In [2]:
import cv2
import mediapipe as mp
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def main(duration=30):
    cap = cv2.VideoCapture(0)
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    start_time = time.time()
    sprint_count = 0
    jump_count = 0

    left_foot_y_prev = None
    right_foot_y_prev = None
    hip_y_prev = None

    step_state = {"left": False, "right": False}
    jump_state = False

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Get foot and hip y-coordinates
            left_foot_y = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y
            right_foot_y = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y
            hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP].y

            # Sprint detection (step alternation)
            if left_foot_y_prev is not None:
                if left_foot_y < left_foot_y_prev - 0.02 and not step_state["left"]:
                    sprint_count += 1
                    step_state["left"] = True
                elif left_foot_y > left_foot_y_prev:
                    step_state["left"] = False

            if right_foot_y_prev is not None:
                if right_foot_y < right_foot_y_prev - 0.02 and not step_state["right"]:
                    sprint_count += 1
                    step_state["right"] = True
                elif right_foot_y > right_foot_y_prev:
                    step_state["right"] = False

            # Jump detection (hip movement)
            if hip_y_prev is not None:
                if hip_y < hip_y_prev - 0.03 and not jump_state:
                    jump_count += 1
                    jump_state = True
                elif hip_y > hip_y_prev:
                    jump_state = False

            # Update previous positions
            left_foot_y_prev = left_foot_y
            right_foot_y_prev = right_foot_y
            hip_y_prev = hip_y

            # Draw landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show counters
        cv2.putText(frame, f"Sprints: {sprint_count}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Jumps: {jump_count}", (30, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        elapsed = time.time() - start_time
        cv2.putText(frame, f"Time: {int(elapsed)}s / {duration}s", (30, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        cv2.imshow("Sprint and Jump Counter", frame)

        # End after duration or on 'q'
        if elapsed >= duration or cv2.waitKey(5) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pose.close()

    print(f"Final Counts -> Sprints: {sprint_count}, Jumps: {jump_count}")

if __name__ == "__main__":
    main(duration=30)  # Run for 30 seconds


Final Counts -> Sprints: 27, Jumps: 4
